In [1]:
import pandas as pd
import os
from datetime import datetime, timedelta
import numpy as np
import math

In [2]:
sp500_url = "https://raw.githubusercontent.com/saikr789/stock-index-risk/master/SP500companies.csv"

In [3]:
# sp500 = pd.read_csv(sp500_url)
# sp500.to_csv("C:\\Users\\venu\\Desktop\\Stock Market Analysis\\Data\\SP500companies.csv", index=False)
sp500 = pd.read_csv("C:\\Users\\venu\\Desktop\\Stock Market Analysis\\Data\\SP500companies.csv")
sp500_sc = list(sp500['Security Code'])
sp500_sn = list(sp500['Security Name'])
sp500_companies = {str(sp500_sc[i]) : sp500_sn[i] for i in range(len(sp500_sc))}
# sp500_companies

In [4]:
def calculate_profit(data, end_date, days_to_search, investment, final_df, final_df1, sec_num):
    start_date = end_date - timedelta(days = days_to_search)
#     print(start_date, end_date)
    mask = (data["date"] >= start_date) & (data["date"] <= end_date)
    data = data.loc[mask]
    data = data.reset_index(drop=True)
    actual_inv = investment
    actual_shares = 0
    pred_inv = investment
    pred_shares = 0
    i = 0
    inv_status = False
    ext_status = False
    actual_first_inv = 0
    pred_first_inv = 0
    pred_ub = 0
    while (i <= data.shape[0] - 1):
#         print(data.loc[i, "date"])
        if (not inv_status):
            if (data.loc[i, "invest"]):
                actual_shares = actual_inv // data.loc[i, "close"]
                actual_first_inv = actual_inv
                actual_inv = 0
                pred_shares = pred_inv // data.loc[i, "close"]
                pred_first_inv = pred_inv
                pred_inv = 0
                pred_ub = data.loc[i, "predicted ub"] * data.loc[i, "close"]
                inv_status = True
                ext_status = False
                row = {}
                row["Date"] = data.loc[i, "date"]
                row["Close Price"] = data.loc[i, "close"]
                row["Invest/Exit"] = "INVEST"
                row["Actual Investment"] = actual_inv
                row["Actual Shares"] = actual_shares
                row["Actual Returns %"] = 0
                row["Predicted Investment"] = pred_inv
                row["Predicted Shares"] = pred_shares
                row["Predicted Value-UB"] = pred_ub
                row["Predicted Returns %"] = 0
                final_df = final_df.append(row, ignore_index=True)
        else:
            if (data.loc[i, "exit"]):
                actual_inv = actual_shares * data.loc[i, "close"]
                actual_shares = 0
                pred_inv = pred_shares * pred_ub
                pred_shares = 0
                ext_status = True
                inv_status = False
                actual_avg = (actual_inv - actual_first_inv) / actual_first_inv
                pred_avg = (pred_inv - pred_first_inv) / pred_first_inv
                row = {}
                row["Date"] = data.loc[i, "date"]
                row["Close Price"] = data.loc[i, "close"]
                row["Invest/Exit"] = "EXIT"
                
                row["Actual Investment"] = actual_inv
                row["Actual Shares"] = actual_shares
                row["Actual Returns %"] = actual_avg
                row["Predicted Investment"] = pred_inv
                row["Predicted Shares"] = pred_shares
                row["Predicted Value-UB"] = 0
                row["Predicted Returns %"] = pred_avg
                
                final_df = final_df.append(row, ignore_index=True)
        i += 1
        
    
    if ((not inv_status) and (not ext_status)):
        row = {}
        row["Date"] = data.loc[data.shape[0] - 1, "date"]
        row["Close Price"] = data.loc[data.shape[0] - 1, "close"]
        row["Invest/Exit"] = "-"
        
        row["Actual Investment"] = 0
        row["Actual Shares"] = 0
        row["Actual Returns %"] = 0
        row["Predicted Investment"] = 0
        row["Predicted Shares"] = 0
        row["Predicted Value-UB"] = 0
        row["Predicted Returns %"] = 0
        
        final_df = final_df.append(row, ignore_index=True)
    
    if (inv_status and (not ext_status)):
        actual_inv = actual_shares * data.loc[data.shape[0] - 1, "close"]
        actual_avg = (actual_inv - actual_first_inv) / actual_first_inv
        actual_shares = 0
        pred_inv = pred_shares * pred_ub
        pred_avg = (pred_inv - pred_first_inv) / pred_first_inv
        pred_shares = 0
        row = {}
        row["Date"] = data.loc[data.shape[0] - 1, "date"]
        row["Close Price"] = data.loc[data.shape[0] - 1, "close"]
        row["Invest/Exit"] = "-"
        
        
        row["Actual Investment"] = actual_inv
        row["Actual Shares"] = actual_shares
        row["Actual Returns %"] = actual_avg
        row["Predicted Investment"] = pred_inv
        row["Predicted Shares"] = pred_shares
        row["Predicted Value-UB"] = pred_ub
        row["Predicted Returns %"] = pred_avg
        
        final_df = final_df.append(row, ignore_index=True)
    
    
#     print(final_df)
    
    actual_total_avg = final_df['Actual Returns %'].sum() / final_df[final_df["Invest/Exit"].isin(["EXIT", "-"])].shape[0]
    pred_total_avg = final_df['Predicted Returns %'].sum() / final_df[final_df["Invest/Exit"].isin(["EXIT", "-"])].shape[0]
    pred_actual_diff = pred_total_avg - actual_total_avg
    company = sp500_companies[sec_num]
    r = {}
    r["Company"] = company
    r["Security Id"] = sec_num
    r["Days"] = days_to_search
    r["Actual average returns %"] = actual_total_avg
    r["Predicted average returns %"] = pred_total_avg
    r["Difference"] = pred_actual_diff
#     r["Days"] = days_to_search
    final_df1 = final_df1.append(r, ignore_index=True)
    
    return actual_inv, actual_shares, pred_inv, pred_shares, final_df, final_df1
        
            

In [5]:
path = os.getcwd()
path

'C:\\Users\\venu\\Desktop\\Stock Market Analysis'

In [6]:
%%time


def get_simulation_loop(days):
    columns1 = ["Company", "Security Id", "Days", "Actual average returns %", "Predicted average returns %", "Difference"]
    fd_df1 = pd.DataFrame(columns = columns1)
    
    for filename in sp500_sc:
#     for filename in os.listdir(os.path.join(path,"Data/SP500_simulation_input_files/simulation")):
#         if (filename != "541450.csv"):
#             continue
        try :
    #         columns = ["Date", "Close Price", "Invest/Exit", "Actual Investment", "Actual Shares", "Actual Returns %", "Predicted Investment", "Predicted Shares", "Predicted UB", "Predicted Returns %"]
    #         fd_df = pd.DataFrame(columns = columns)
            print("-------------------------------------------------------")
            name = str(filename)
#             name = filename[ : 6]
            print(name)
            df1 = pd.read_csv("https://raw.githubusercontent.com/saikr789/stock-analysis-tool-1011-data/master/Data/Simulation/" + str(filename) + "_" + str(days) + ".csv")
#             df1 = pd.read_csv("https://raw.githubusercontent.com/saikr789/stock-index-risk/master/Data/Simulation/" + str(filename) + ".csv")
#             df1 = pd.read_csv("C:\\Users\\venu\\Desktop\\Stock Market Analysis\\Data\\SP500_simulation_input_files\\simulation\\" + filename)
            df1["date"] = pd.to_datetime(df1["date"])
            st_date = df1.loc[0, "date"] + timedelta(days = days)
            j = 1
            
#             print("before while", st_date, df1.loc[df1.shape[0] - j, "date"])
            
            while (st_date <= (df1.loc[df1.shape[0] - j, "date"])):
                columns = ["Date", "Close Price", "Invest/Exit", "Actual Investment", "Actual Shares", "Actual Returns %", "Predicted Investment", "Predicted Shares", "Predicted Value-UB", "Predicted Returns %"]
                fd_df = pd.DataFrame(columns = columns)
                ed_date = df1.loc[df1.shape[0] - j, "date"]
#                 print(st_date, ed_date, "while")
                inst = 100000
                actual_investment, actual_shares, predicted_investment, predicted_shares, final_df, fd_df1 = calculate_profit(df1, ed_date, days, inst, fd_df, fd_df1, name)
    #             print(actual_investment, actual_shares)
    #             print(predicted_investment, predicted_shares)
        #         print(final_df)
        #         print(fd_df1)
    #             final_df.to_csv("C:\\Users\\venu\\Desktop\\Stock Market Analysis\\Data\\SP500_simulation_results\\" + name + "_" + str(days) + "_days_" + str(j) + ".csv", index = False)
    #             print("-------------------------------------------------------")
                j += 1
#             break
            print("-------------------------------------------------------")
        except Exception as e:
            print(e)
            continue

    fd_df1 = fd_df1.sort_values(by = ['Security Id'], ascending = [True])
    print(fd_df1)
#     fd_df1.to_csv("C:\\Users\\venu\\Desktop\\Stock Market Analysis\\Data\\SP500_simulation_results\\all_" + str(days) + "days_Results.csv", index = False)



Wall time: 0 ns


In [7]:
%%time

days = [30, 60, 90, 180, 360, 720, 1080]

for d in days:
#     if(d != 1080):
#         continue
    print(d)
    print("*****************************************************************")
    get_simulation_loop(d)
    print("*****************************************************************")
#     break

1080
*****************************************************************
-------------------------------------------------------
500002
before while 2019-01-20 00:00:00 2018-06-08 00:00:00
Empty DataFrame
Columns: [Company, Security Id, Days, Actual average returns %, Predicted average returns %, Difference]
Index: []
*****************************************************************
Wall time: 1.24 s
